In [ ]:
import os
from IPython.display import Image
PATH = "C:\\Users\\Hakan\\Desktop\\Covid-19\\"
Image(filename = PATH + "CVP.jpg", width=900, height=900)

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import math
import random
import json, requests
import plotly.express as px
import plotly.graph_objs as go
import pycountry_convert as pc
import folium
import branca
import calmap
import glob
import time
import datetime
#import timedelta
#import date
import operator
import re
import requests
import wget

from bs4 import BeautifulSoup

from math import sqrt
from matplotlib import pyplot as plt
from matplotlib import ticker 
from matplotlib import colors as mcolors

from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error

#from datetime import datetime, timedelta, date

from urllib.request import urlopen

from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()   

import warnings
warnings.filterwarnings('ignore')

init_notebook_mode(connected=True)

sns.set_style('darkgrid')

plt.style.use('fivethirtyeight')

pd.set_option('display.max_rows', None)

%matplotlib inline

np.set_printoptions(formatter={'float_kind':'{:f}'.format})

sns.set(rc={'figure.figsize':(12,12)})

In [ ]:
link = 'https://www.worldometers.info/coronavirus/'

req = requests.get(link)

soup = BeautifulSoup(req.content, "html.parser")

In [ ]:
thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

In [ ]:
head_rows = []

body_rows = []

for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)

In [ ]:
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

df_bs.head(10)

In [ ]:
df_bs.tail()

In [ ]:
df_bs = df_bs.loc[8:].reset_index(drop=True)
df_bs = df_bs.drop('#', axis=1)
df_bs.head()

In [ ]:
df_bs = df_bs.iloc[:, :-3]

In [ ]:
df_bs.columns

In [ ]:
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

df_bs.head()

In [ ]:
df_bs.columns[2:]

In [ ]:
for col in df_bs.columns[2:]:
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)
 

df_bs.replace('', np.nan)

In [ ]:
df_bs[df_bs['Country/Region'] == 'Turkey']

In [ ]:
df_bs.to_csv('worldometer_data.csv', index=False)

In [ ]:
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']
    
for url in urls:
    filename = wget.download(url)

In [ ]:
raw_data_confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
raw_data_deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
raw_data_recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
worldwide = pd.read_csv("https://raw.githubusercontent.com/datasets/covid-19/master/data/worldwide-aggregated.csv")

print("The Shape of Confirmed is: ", raw_data_confirmed.shape)
print("The Shape of Death is: ", raw_data_deaths.shape)
print("The Shape of Recovered is: ", raw_data_recovered.shape)

raw_data_confirmed.head()

In [ ]:
raw_data_confirmed[raw_data_confirmed['Country/Region'] == 'Turkey']

In [ ]:
raw_data_deaths[raw_data_deaths['Country/Region'] == 'Turkey']

In [ ]:
raw_data_recovered[raw_data_recovered['Country/Region'] == 'Turkey']

In [ ]:
raw_data_confirmed2 = pd.melt(raw_data_confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_deaths2 = pd.melt(raw_data_deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_recovered2 = pd.melt(raw_data_recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])

print("The Shape of Confirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Death is: ", raw_data_deaths2.shape)
print("The Shape of Recovered is: ", raw_data_recovered2.shape)

raw_data_confirmed2.head()

In [ ]:
raw_data_confirmed2['Date'] = pd.to_datetime(raw_data_confirmed2['Date'])
raw_data_deaths2['Date'] = pd.to_datetime(raw_data_deaths2['Date'])
raw_data_recovered2['Date'] = pd.to_datetime(raw_data_recovered2['Date'])
worldwide['Date'] = pd.to_datetime(worldwide['Date'])

In [ ]:
raw_data_confirmed2.columns = raw_data_confirmed2.columns.str.replace('value', 'Confirmed')
raw_data_deaths2.columns = raw_data_deaths2.columns.str.replace('value', 'Deaths')
raw_data_recovered2.columns = raw_data_recovered2.columns.str.replace('value', 'Recovered')

In [ ]:
raw_data_recovered2.isnull().sum()

In [ ]:
raw_data_confirmed2['Province/State'].fillna(raw_data_confirmed2['Country/Region'], inplace=True)
raw_data_deaths2['Province/State'].fillna(raw_data_deaths2['Country/Region'], inplace=True)
raw_data_recovered2['Province/State'].fillna(raw_data_recovered2['Country/Region'], inplace=True)

raw_data_confirmed2.isnull().sum()

In [ ]:
print("The Shape of Confirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Death is: ", raw_data_deaths2.shape)
print("The Shape of Recovered is: ", raw_data_recovered2.shape)

In [ ]:
raw_data_confirmed2.isnull().sum()
raw_data_deaths2.isnull().sum()
raw_data_recovered2.isnull().sum()

In [ ]:
full_join = raw_data_confirmed2.merge(raw_data_deaths2[['Province/State','Country/Region','Date','Deaths']],
how = 'left',
left_on = ['Province/State','Country/Region','Date'],
right_on = ['Province/State','Country/Region','Date'])

print("Shape of first join: ", full_join.shape)

full_join = full_join.merge(raw_data_recovered2[['Province/State','Country/Region','Date','Recovered']],
how = 'left',
left_on = ['Province/State','Country/Region','Date'],
right_on = ['Province/State','Country/Region','Date'])

print("Shape of second join: ", full_join.shape)

full_join.head()

In [ ]:
full_join['Active Cases'] = full_join['Confirmed'] - full_join['Deaths'] - full_join['Recovered']
worldwide['Active Cases'] = worldwide['Confirmed'] - worldwide['Deaths'] - worldwide['Recovered']

In [ ]:
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%y')
worldwide['Month-Year'] = worldwide['Date'].dt.strftime('%b-%y')

In [ ]:
full_join.fillna(0 , inplace=True)
worldwide.fillna(0 , inplace=True)

In [ ]:
full_join[["Recovered"]] = full_join[["Recovered"]].astype(int)
worldwide[["Recovered"]] = worldwide[["Recovered"]].astype(int)

In [ ]:
full_join.isnull().sum()

In [ ]:
worldwide.isnull().sum()

In [ ]:
worldwide

In [ ]:
full_join2 = full_join.copy()
worldwide2 = worldwide.copy()



full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                           'Date': 'Date Minus 1'}, inplace=True)


worldwide2['Date - 1'] = worldwide2['Date'] + pd.Timedelta(days=1)
worldwide2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                           'Date': 'Date Minus 1'}, inplace=True)


full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region', 'Confirmed - 1', 'Deaths - 1', 'Recovered - 1',
                                         'Date - 1', 'Date Minus 1']], how='left', 
                                left_on=['Province/State', 'Country/Region', 'Date'],
                               right_on=['Province/State', 'Country/Region', 'Date - 1'])


worldwide3 = worldwide.merge(worldwide2[['Confirmed - 1', 'Deaths - 1', 'Recovered - 1',
                                         'Date - 1', 'Date Minus 1']], how='left', 
                                left_on=['Date'],
                               right_on=['Date - 1'])


full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths - 1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered - 1']


worldwide3['Confirmed Daily'] = worldwide3['Confirmed'] - worldwide3['Confirmed - 1']
worldwide3['Recovered Daily'] = worldwide3['Recovered'] - worldwide3['Recovered - 1']
worldwide3['Deaths Daily'] = worldwide3['Deaths'] - worldwide3['Deaths - 1']


print(full_join3.shape)
print(worldwide3.shape)

In [ ]:
full_join3.head()

In [ ]:
worldwide3.head()

In [ ]:
full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
full_join3['Recovered Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Recovered']

worldwide3['Confirmed Daily'].loc[worldwide3['Date'] == '2020-01-22'] = worldwide3['Confirmed']
worldwide3['Recovered Daily'].loc[worldwide3['Date'] == '2020-01-22'] = worldwide3['Recovered']
worldwide3['Deaths Daily'].loc[worldwide3['Date'] == '2020-01-22'] = worldwide3['Deaths']

del full_join3['Confirmed - 1']
del full_join3['Deaths - 1']
del full_join3['Recovered - 1']
del full_join3['Date - 1']
del full_join3['Date Minus 1']

del worldwide3['Confirmed - 1']
del worldwide3['Deaths - 1']
del worldwide3['Recovered - 1']
del worldwide3['Date - 1']
del worldwide3['Date Minus 1']

In [ ]:
full_join3.head()

In [ ]:
worldwide3.head()

In [ ]:
full_join3[["Confirmed Daily","Deaths Daily","Recovered Daily","Active Cases"]] = full_join3[["Confirmed Daily","Deaths Daily","Recovered Daily","Active Cases"]].astype(int)
worldwide3[["Confirmed Daily","Deaths Daily","Recovered Daily","Active Cases"]] = worldwide3[["Confirmed Daily","Deaths Daily","Recovered Daily","Active Cases"]].astype(int)

In [ ]:
full_join3['Turkey vs Rest of the World'] = 'Rest of the World'
full_join3['Turkey vs Rest of the World'].loc[full_join3['Province/State'] == 'Turkey'] = 'Turkey - Virus'

In [ ]:
full_join3[full_join3['Country/Region'] == 'Turkey']

In [ ]:
full_join3.head()

In [ ]:
turkey_province_wise = full_join3[full_join3['Country/Region']=='Turkey']
turkey_province_wise['Province/State'].unique()
turkey_province_wise.to_csv('turkey_province_wise.csv', sep='\t', index=False)

In [ ]:
us_province_wise = full_join3[full_join3['Country/Region']=='US']
us_province_wise['Province/State'].unique()
us_province_wise.to_csv('us_province_wise.csv', sep='\t', index=False)

In [ ]:
canada_province_wise = full_join3[full_join3['Country/Region']=='Canada']
canada_province_wise['Province/State'].unique()
canada_province_wise.to_csv('canada_province_wise.csv', sep='\t', index=False)

In [ ]:
brazil_province_wise = full_join3[full_join3['Country/Region']=='Brazil']
brazil_province_wise['Province/State'].unique()
brazil_province_wise.to_csv('brazil_province_wise.csv', sep='\t', index=False)

In [ ]:
mexico_province_wise = full_join3[full_join3['Country/Region']=='Mexico']
mexico_province_wise['Province/State'].unique()
mexico_province_wise.to_csv('mexico_province_wise.csv', sep='\t', index=False)

In [ ]:
peru_province_wise = full_join3[full_join3['Country/Region']=='Peru']
peru_province_wise['Province/State'].unique()
peru_province_wise.to_csv('peru_province_wise.csv', sep='\t', index=False)

In [ ]:
chile_province_wise = full_join3[full_join3['Country/Region']=='Chile']
chile_province_wise['Province/State'].unique()
chile_province_wise.to_csv('chile_province_wise.csv', sep='\t', index=False)

In [ ]:
russia_province_wise = full_join3[full_join3['Country/Region']=='Russia']
russia_province_wise['Province/State'].unique()
russia_province_wise.to_csv('russia_province_wise.csv', sep='\t', index=False)

In [ ]:
china_province_wise = full_join3[full_join3['Country/Region']=='China']
china_province_wise['Province/State'].unique()
china_province_wise.to_csv('china_province_wise.csv', sep='\t', index=False)

In [ ]:
india_province_wise = full_join3[full_join3['Country/Region']=='India']
india_province_wise['Province/State'].unique()
india_province_wise.to_csv('india_province_wise.csv', sep='\t', index=False)

In [ ]:
pakistan_province_wise = full_join3[full_join3['Country/Region']=='Pakistan']
pakistan_province_wise['Province/State'].unique()
pakistan_province_wise.to_csv('pakistan_province_wise.csv', sep='\t', index=False)

In [ ]:
iran_province_wise = full_join3[full_join3['Country/Region']=='Iran']
iran_province_wise['Province/State'].unique()
iran_province_wise.to_csv('iran_province_wise.csv', sep='\t', index=False)

In [ ]:
uk_province_wise = full_join3[full_join3['Country/Region']=='United Kingdom']
uk_province_wise['Province/State'].unique()
uk_province_wise.to_csv('uk_province_wise.csv', sep='\t', index=False)

In [ ]:
italy_province_wise = full_join3[full_join3['Country/Region']=='Italy']
italy_province_wise['Province/State'].unique()
italy_province_wise.to_csv('italy_province_wise.csv', sep='\t', index=False)

In [ ]:
spain_province_wise = full_join3[full_join3['Country/Region']=='Spain']
spain_province_wise['Province/State'].unique()
spain_province_wise.to_csv('spain_province_wise.csv', sep='\t', index=False)

In [ ]:
france_province_wise = full_join3[full_join3['Country/Region']=='France']
france_province_wise['Province/State'].unique()
france_province_wise.to_csv('france_province_wise.csv', sep='\t', index=False)

In [ ]:
germany_province_wise = full_join3[full_join3['Country/Region']=='Germany']
germany_province_wise['Province/State'].unique()
germany_province_wise.to_csv('germany_province_wise.csv', sep='\t', index=False)

In [ ]:
netherlands_province_wise = full_join3[full_join3['Country/Region']=='Netherlands']
netherlands_province_wise['Province/State'].unique()
netherlands_province_wise.to_csv('netherlands_province_wise.csv', sep='\t', index=False)

In [ ]:
path = "C:\\Users\\Hakan\\Desktop\\Covid-19\\"

os.chdir(path)

worldwide3.to_csv('Worldwide PowerBI.csv', sep='\t', index=False)

In [ ]:
path = "C:\\Users\\Hakan\\Desktop\\Covid-19\\"

os.chdir(path)

full_join3.to_csv('CoronaVirus PowerBI.csv', sep='\t', index=False)